### Assignment 6
    Akhil Kanna Devarashetti

Question:

    Write a Pytorch version of the Word2vec/skip-gram displayed in Chapter 14 of d2l.  
    In particular, make DL24.py error free 
    Implement get_similar_tokens as an application of the word embedding model 
    (14.4.3 of d2l and also the last slide in lecture)

In [ ]:
# DL24.py CS5173/6073 2020 cheng
# making centers, contexts, and negatives for PennTreebank data
# building vocabulary, performing subsampling and negative sampling
# Skip-gram word embedding as a translation from MXNet to Pytorch of d2l chapter 14
# Usage: python DL24.py

import zipfile
import collections
import random
import math
import torch
import numpy as np

f = zipfile.ZipFile('data/ptb.zip', 'r')
raw_text = f.read('ptb/ptb.train.txt').decode("utf-8")
sentences = [line.split() for line in raw_text.split('\n')]
tokens = [tk for line in sentences for tk in line]
counter = collections.Counter(tokens)
uniq_tokens = [token for token, freq in list(counter.items()) if counter[token] >= 10]
idx_to_token, token_to_idx = [], dict()
for token in uniq_tokens:
    idx_to_token.append(token)
    token_to_idx[token] = len(idx_to_token) - 1
s = [[idx_to_token[token_to_idx.get(tk, 0)] for tk in line] for line in sentences]
tokens = [tk for line in s for tk in line]
counter = collections.Counter(tokens)
num_tokens = sum(counter.values())
subsampled = [[tk for tk in line if random.uniform(0, 1) < math.sqrt(1e-4 / counter[tk] * num_tokens)] for line in s]
corpus = [[token_to_idx.get(tk) for tk in line] for line in subsampled]
tokens = [tk for line in corpus for tk in line]
counter = collections.Counter(tokens)
sampling_weights = [counter[i]**0.75 for i in range(len(counter))]
population = list(range(len(sampling_weights)))
candidates = random.choices(population, sampling_weights, k=10000)
max_window_size = 5
K = 5
j = 0
data = []
maxLen = 0
for line in corpus:
    if len(line) < 2:
        continue
    for i in range(len(line)):
        window_size = random.randint(1, max_window_size)
        indices = list(range(max(0, i - window_size), min(len(line), i + 1 + window_size)))
        indices.remove(i)
        for idx in indices:
            context = [line[idx] for idx in indices]
        neg = []
        while len(neg) < len(context) * K:
            ne = candidates[j]
            j += 1
            if j >= 10000:
                j = 0
            if ne not in context:
                neg.append(ne)
        data.append([line[i], context, neg])

max_len = max(len(c) + len(n) for _, c, n in data)
centers, contexts_negatives, labels = [], [], []
for center, context, negative in data:
    cur_len = len(context) + len(negative)
    centers += [center]
    contexts_negatives += [context + negative + [0] * (max_len - cur_len)]
    labels += [[1] * len(context) + [0] * (max_len - len(context))]

class PTBdataset(torch.utils.data.Dataset):
    def __init__(self):
        super(PTBdataset).__init__()
        self.centers = np.array(centers).reshape(-1, 1)
        self.contexts_negatives = np.array(contexts_negatives)
        self.labels = np.array(labels)

    def __len__(self):
        return len(self.centers)

    def __getitem__(self, idx):
        return self.centers[idx], self.contexts_negatives[idx], self.labels[idx]

pdata = PTBdataset()
data_iter = torch.utils.data.DataLoader(pdata, batch_size=512, shuffle=True)

vocab_size = len(idx_to_token)
embed_size = 100

import torch.nn as nn
import torch.optim as optim

net = nn.Sequential(
    nn.Embedding(vocab_size, embed_size),
    nn.Embedding(vocab_size, embed_size))
loss = nn.BCELoss()
optimizer = optim.SGD(net.parameters(), 0.01)
m = nn.Sigmoid()

for epoch in range(5):
    for i, batch in enumerate(data_iter):
        center, context_negative, label = batch
        v = net[0](center.to(torch.int64))
        u = net[1](context_negative.to(torch.int64))
        pred = torch.tensordot(v, torch.transpose(u, 1, 2))
        l = loss(m(pred), label.to(torch.float32))
        optimizer.zero_grad()
        l.backward()
        optimizer.step()
        if (i + 1) % 50 == 0:
            print(epoch, i, float(l))



In [ ]:
def get_similar_tokens(query_token, k, embed):
    W = embed.weight.data
    x = W[token_to_idx[query_token]]
    # Compute the cosine similarity. Add 1e-9 for numerical stability
    cos = torch.matmul(W, x) / (
        (torch.sum(W * W, dim=1) + 1e-9).sqrt() * torch.sum(x * x).sqrt())
    _, topk = torch.topk(cos, k=k)
    topk = topk.cpu().numpy()
    for i in topk[1:]:  # Remove the input words
        print('cosine sim=%.3f: %s' % (cos[i], (idx_to_token[i])))

get_similar_tokens('chip', 3, net[0])

In [ ]:
def knn(W, x, k):
    cos = torch.matmul(W, x) / (
        (torch.sum(W * W, dim=1) + 1e-9).sqrt() * torch.sum(x * x).sqrt())
    _, topk = torch.topk(cos, k=k)
    topk = topk.cpu().numpy()
    return topk, [cos[i].item() for i in topk]

def get_similar_tokens(query_token, k, embed):
    W = embed.weight.data
    x = W[token_to_idx[query_token]]
    topk, cos = knn(W, x, k+1)
    for i, c in zip(topk[1:], cos[1:]):
        print('cosine sim=%.3f: %s' % (c, (idx_to_token[i])))

get_similar_tokens('chip', 3, net[0])

In [ ]:
words_for_similarity = ['chip', 'hardware']

for word in words_for_similarity:
    print(f"\nSimilarity for '{word}':")
    get_similar_tokens(word, 3, net[0])


In [ ]:
embed = net[0]
query_token = 'chip'
W = embed.weight.data
x = W[token_to_idx[query_token]]


In [ ]:
print(W.shape)
print(x.shape)
matmuled = torch.matmul(W, x)
print(matmuled.shape)
cos = torch.matmul(W, x) / (
        (torch.sum(W * W, dim=1) + 1e-9).sqrt() * torch.sum(x * x).sqrt())
print(cos.shape)

In [ ]:
input1 = torch.randn(2, 2)
input2 = torch.randn(1, 2)
print(input1.shape)

cos = nn.CosineSimilarity(dim=1, eps=1e-6)
output = cos(input1, input2)

print(output.shape)

In [ ]:
def get_similar_tokens(query_token, k, embed):
    W = embed.weight.data
    x = W[token_to_idx[query_token]]
    x = x.reshape([1] + list(x.shape))

    # Compute the cosine similarity. Add 1e-9 for numerical stability
    cos_similarity = nn.CosineSimilarity(dim=1, eps=1e-6)
    cos = cos_similarity(W, x)

    _, topk = torch.topk(cos, k=k)
    topk = topk.cpu().numpy()
    for i in topk[1:]:  # Remove the input words
        print('cosine sim=%.3f: %s' % (cos[i], (idx_to_token[i])))

get_similar_tokens('chip', 3, net[0])
